In [2]:
def converter(input_files, output_file, station_list_file):
    event_counter = 0  # Initialize an event counter

    # Step 1: Read station names from the text file into a list
    with open(station_list_file, 'r') as station_file:
        allowed_stations = [line.strip() for line in station_file]

    # Step 2: Open the output file for writing
    with open(output_file, 'w') as outfile:
        # Loop through each input file
        for fileinput in input_files:
            with open(fileinput, 'r') as infile:
                # Read all lines from the input file and split each line into a list of words
                lines = infile.readlines()
                for i in range(len(lines)):
                    lines[i] = lines[i].split()

                i = 0
                while i < len(lines):
                    if len(lines[i]) > 0 and lines[i][0] == 'EventID:':
                        # Extract information from the seismic event record
                        year = lines[i + 2][0].split('-')[0]
                        month = lines[i + 2][0].split('-')[1].zfill(2)
                        date = lines[i + 2][0].split('-')[2].zfill(2)
                        hour = lines[i + 2][1].split(':')[0].zfill(2)
                        minute = lines[i + 2][1].split(':')[1].zfill(2)
                        second = (('%.1f') % float(lines[i + 2][1].split(':')[2])).zfill(4)
                        latitude = (('%.2f') % float(lines[i + 2][2])).zfill(6)
                        longitude = (('%.2f') % float(lines[i + 2][3])).zfill(6)
                        
                        # Apply a filter based on latitude and longitude
                        if -13 <= float(latitude) <= -4 and 110 <= float(longitude) < 113.0:
                            event_counter += 1
                            depth = ('%.1f') % float(lines[i + 2][4])
                            mag = ('%.1f') % float(lines[i + 2][5])
                            unknown = '0.0'
                            rms = ('%.3f') % float(lines[i + 2][10])
                            time0 = float(second) + float(minute) * 60 + float(hour) * 60 * 60 + float(date) * 60 * 60 * 24
                            
                            # Write the extracted and filtered information to the output file
                            outfile.write('#' + year.rjust(5) + month.rjust(3) + date.rjust(3) + hour.rjust(3) +
                                          minute.rjust(3) + second.rjust(5) + latitude.rjust(8) + longitude.rjust(10) +
                                          depth.rjust(9) + mag.rjust(5) + unknown.rjust(6) + unknown.rjust(6) +
                                          rms.rjust(8) + str(event_counter).rjust(10) + '\n')

                    if len(lines[i]) > 0 and lines[i][0] == 'Net' and -13 <= float(latitude) <= -4 and 110 <= float(longitude) < 113.0: # apply the filter in this line too
                        try:
                            j = 0
                            while j < 1:
                                i = i + 1
                                try:
                                    station = lines[i][1]
                                    if station in allowed_stations:
                                        phase = lines[i][2]
                                        date = lines[i][3].split('-')[2]
                                        hour = lines[i][4].split(':')[0]
                                        minute = lines[i][4].split(':')[1]
                                        second = lines[i][4].split(':')[2]
                                        time1 = float(second) + float(minute) * 60 + float(hour) * 60 * 60 + float(date) * 60 * 60 * 24
                                        deltatime = ('%.2f') % (time1 - time0)
                                        unknown = '1.000'
                                        
                                        # Write station-specific information to the output file
                                        outfile.write(station.rjust(5) + deltatime.rjust(12) + unknown.rjust(8) +
                                                      phase.rjust(4) + '\n')
                                except:
                                    break
                        except:
                            break
                    i = i + 1


In [3]:
# Example usage:
input_files = ['2012.txt','2013.txt','2014.txt','2015.txt','2016.txt','2017.txt']  # Add your input file paths here
output_file = '2012-2017_Kel5AB.pha'
station_filter = 'filter_station.txt'
converter(input_files, output_file, station_filter)

In [4]:
def edit_event_phase(phase, header, output_file):
    with open(phase, 'r') as f1, open(header, 'r') as f2, open(output_file, 'w') as output:
        data = f1.readlines()
        for i, line in enumerate(data):
            if '#' in line and len(line.split()) > 5:
                output.write(next(f2).strip() + '\n')
            else:
                output.write(f"{line.split()[0]}   {line.split()[1]}    {line.split()[2]}   {line.split()[3]}\n")



In [5]:
# Usage example:
phase = '2012-2017_Kel5AB.pha'
header = 'new_header.txt'
output_file ='absolute.dat'
edit_event_phase(phase, header, output_file)

In [6]:
with open('2012-2017_Kel5AB.pha', 'r') as f:
    lines = f.readlines()

new_lines = []
for i, line in enumerate(lines):
    if line.startswith('#'):
        new_lines.append(line)
        continue
    station = line.split()[0]
    event_lines = [line]
    for next_line in lines[i+1:]:
        if next_line.startswith('#'):
            break
        next_station = next_line.split()[0]
        if next_station == station:
            event_lines.append(next_line)
    has_p = False
    has_s = False
    if len(event_lines) == 2:
        for event_line in event_lines:
            if event_line.split()[3] == 'P':
                has_p = True
            elif event_line.split()[3] == 'S':
                has_s = True
        if has_p and has_s:
            new_lines.extend(event_lines)
    elif len(event_lines) == 4:
        new_lines.extend(event_lines)

for elem in new_lines:
    print(elem)

# 2012 01 01 18 01 30.4  -07.94    111.65     10.0  2.6   0.0   0.0   0.817         1

 SWJI        5.80   1.000   P

 SWJI        9.40   1.000   S

 PWJI        7.10   1.000   P

 PWJI       11.40   1.000   S

 PCJI       11.70   1.000   P

 PCJI       19.20   1.000   S

# 2012 01 02 18 39 51.9  -08.87    111.45     28.0  2.9   0.0   0.0   1.113         2

 PCJI       13.80   1.000   P

 PCJI       23.60   1.000   S

 PWJI       15.80   1.000   P

 PWJI       27.30   1.000   S

 SWJI       20.50   1.000   P

 SWJI       35.50   1.000   S

# 2012 01 03 14 11 50.7  -09.81    112.97     18.0  3.4   0.0   0.0   0.331         3

 GMJI       27.20   1.000   P

 GMJI       47.80   1.000   S

 PWJI       33.30   1.000   P

 PWJI       57.50   1.000   S

# 2012 01 03 19 12 22.5  -09.11    111.79    109.0  2.7   0.0   0.0   0.624         4

 PWJI       20.40   1.000   P

 PWJI       37.30   1.000   S

 PCJI       21.10   1.000   P

 PCJI       38.80   1.000   S

 SWJI       26.20   1.000   P

 